# Automated Backend Deployment with Ngrok

This notebook deploys the backend service on port 5000 using ngrok in Google Colab.

It handles:
1. Cloning the repository
2. Installing dependencies
3. Setting up ngrok tunnel
4. Starting the backend server
5. Monitoring and managing processes

## Setup Environment

In [ ]:
# Install ngrok
!pip install pyngrok
!pip install psutil

# Import necessary libraries
import os
import subprocess
import time
import signal
import psutil
from pyngrok import ngrok, conf
import json
import requests
from IPython.display import clear_output, display, HTML
from google.colab import files
import sys

## Clone Repository

In [ ]:
# Clone your repository (update with your actual repository URL)
REPO_URL = "https://github.com/yourusername/your-repo.git"  # CHANGE THIS TO YOUR REPO URL

# Check if the repository is already cloned
if not os.path.exists('frontend-psg'):
    !git clone $REPO_URL frontend-psg

# Change to the repository directory
%cd frontend-psg

## Install Dependencies

In [ ]:
# Install Python dependencies
!pip install -r requirements.txt

## Process Management Functions

In [ ]:
# Dictionary to store all processes
processes = {}

def get_process_info():
    """Get information about running processes"""
    process_info = []
    for name, proc in processes.items():
        if isinstance(proc, subprocess.Popen):
            try:
                p = psutil.Process(proc.pid)
                status = "Running" if p.is_running() else "Stopped"
                process_info.append({"name": name, "pid": proc.pid, "status": status})
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                process_info.append({"name": name, "pid": proc.pid, "status": "Terminated"})
    return process_info

def kill_process(proc_name):
    """Kill a specific process by name"""
    if proc_name in processes:
        proc = processes[proc_name]
        if isinstance(proc, subprocess.Popen):
            try:
                # Try to terminate gracefully first
                parent = psutil.Process(proc.pid)
                for child in parent.children(recursive=True):
                    child.terminate()
                parent.terminate()
                
                # Give it some time to terminate
                time.sleep(2)
                
                # Force kill if still running
                if parent.is_running():
                    for child in parent.children(recursive=True):
                        child.kill()
                    parent.kill()
                    
                print(f"Process '{proc_name}' (PID: {proc.pid}) has been killed.")
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                print(f"Process '{proc_name}' was already terminated.")
            
            # Remove from our processes dictionary
            del processes[proc_name]
            return True
    print(f"Process '{proc_name}' not found.")
    return False

def kill_all_processes():
    """Kill all tracked processes"""
    process_names = list(processes.keys())
    for proc_name in process_names:
        kill_process(proc_name)
    
    # Also kill any ngrok processes that might be running
    try:
        ngrok.kill()
    except:
        pass
        
    print("All processes have been terminated.")

## Ngrok Tunnel Functions

In [ ]:
def create_ngrok_tunnel(port=5000, authtoken=None):
    """Create an ngrok tunnel to the specified local port"""
    try:
        # If an authtoken is provided, set it
        if authtoken:
            conf.get_default().auth_token = authtoken
        
        # Kill any existing ngrok processes
        ngrok.kill()
        
        # Start a new tunnel - allowing all traffic (tcp and http)
        http_tunnel = ngrok.connect(port, "http")
        tcp_tunnel = ngrok.connect(port, "tcp")
        
        # Get tunnel URLs
        http_url = http_tunnel.public_url
        tcp_url = tcp_tunnel.public_url
        
        # Store tunnel info in a file
        tunnel_info = {
            "http_url": http_url,
            "tcp_url": tcp_url,
            "local_port": port
        }
        
        with open('ngrok_tunnel_info.json', 'w') as f:
            json.dump(tunnel_info, f, indent=2)
        
        return http_url, tcp_url
    except Exception as e:
        print(f"Error creating ngrok tunnel: {str(e)}")
        return None, None

def get_tunnel_info():
    """Get information about all active ngrok tunnels"""
    try:
        tunnels = ngrok.get_tunnels()
        return [{"public_url": tunnel.public_url, "proto": tunnel.proto} for tunnel in tunnels]
    except Exception as e:
        print(f"Error getting tunnel info: {str(e)}")
        return []

## Start Backend Server

In [ ]:
def start_backend_server():
    """Start the Flask backend server"""
    try:
        # Check if process is already running
        if "backend" in processes:
            print("Backend is already running.")
            return False
        
        # Start the backend as a subprocess
        print("Starting the backend server...")
        backend_proc = subprocess.Popen([sys.executable, "app.py"], 
                                        stdout=subprocess.PIPE,
                                        stderr=subprocess.PIPE,
                                        text=True)
        
        # Store the process
        processes["backend"] = backend_proc
        
        # Wait briefly for server to start
        time.sleep(3)
        
        # Check if process is still running
        if backend_proc.poll() is not None:
            # Process has already terminated
            stdout, stderr = backend_proc.communicate()
            print(f"Backend server failed to start:\nStdout: {stdout}\nStderr: {stderr}")
            return False
        
        print(f"Backend server started with PID: {backend_proc.pid}")
        return True
    except Exception as e:
        print(f"Error starting backend server: {str(e)}")
        return False

## Deploy Backend with Ngrok

In [ ]:
def deploy_backend_with_ngrok(ngrok_authtoken=None):
    """Deploy the backend and expose it through ngrok"""
    try:
        # Start the backend server
        if not start_backend_server():
            return False
        
        # Create ngrok tunnel
        http_url, tcp_url = create_ngrok_tunnel(port=5000, authtoken=ngrok_authtoken)
        
        if not http_url or not tcp_url:
            print("Failed to create ngrok tunnels.")
            kill_process("backend")
            return False
        
        print(f"\nBackend is now accessible at:\n")
        print(f"HTTP: {http_url}")
        print(f"TCP: {tcp_url}")
        print(f"\nYour backend API is available at: {http_url}/api/generate-speech")
        
        # Display a clickable link
        display(HTML(f'<a href="{http_url}" target="_blank">Open Backend (HTTP)</a>'))
        
        return True
    except Exception as e:
        print(f"Error deploying backend: {str(e)}")
        # Cleanup in case of failure
        kill_all_processes()
        return False

## Status Monitoring

In [ ]:
def show_deployment_status():
    """Display the current status of all processes and tunnels"""
    clear_output(wait=True)
    
    print("===== DEPLOYMENT STATUS =====\n")
    
    # Show process status
    proc_info = get_process_info()
    if proc_info:
        print("Running Processes:")
        for proc in proc_info:
            print(f"  - {proc['name']} (PID: {proc['pid']}, Status: {proc['status']})")
    else:
        print("No processes are currently running.")
    
    print("\n")
    
    # Show tunnel status
    tunnels = get_tunnel_info()
    if tunnels:
        print("Active Tunnels:")
        for tunnel in tunnels:
            print(f"  - {tunnel['proto']}: {tunnel['public_url']}")
            if tunnel['proto'] == 'http':
                display(HTML(f'<a href="{tunnel["public_url"]}" target="_blank">Open Link</a>'))
    else:
        print("No active tunnels found.")
        
    # Backend API endpoint if running
    if "backend" in processes and any(t['proto'] == 'http' for t in tunnels):
        http_url = next((t['public_url'] for t in tunnels if t['proto'] == 'http'), None)
        if http_url:
            print(f"\nBackend API endpoint: {http_url}/api/generate-speech")
    
    return proc_info, tunnels

## Deployment Controls

In [ ]:
# Deploy backend with ngrok
# You can provide your ngrok authtoken for higher rate limits and longer session duration
# If you don't have one, you can sign up at https://dashboard.ngrok.com/signup

NGROK_AUTHTOKEN = ""  # Optional: Add your ngrok authtoken here or leave empty

deploy_backend_with_ngrok(ngrok_authtoken=NGROK_AUTHTOKEN)

In [ ]:
# Check deployment status
show_deployment_status()

In [ ]:
# Stop deployment
kill_all_processes()

## Test API Endpoint

Once your backend is deployed, you can test the API endpoint with the following:

In [ ]:
def test_api_endpoint():
    """Test the API endpoint with a sample request"""
    # Get tunnel info
    tunnels = get_tunnel_info()
    http_url = next((t['public_url'] for t in tunnels if t['proto'] == 'http'), None)
    
    if not http_url:
        print("No HTTP tunnel found. Please deploy the backend first.")
        return
    
    # Sample request data - update based on your API requirements
    test_data = {
        "prompt": "This is a test message"
    }
    
    # Send test request
    try:
        endpoint = f"{http_url}/api/generate-speech"
        print(f"Sending test request to: {endpoint}")
        response = requests.post(
            endpoint,
            json=test_data,
            headers={"Content-Type": "application/json"}
        )
        
        # Print response
        print(f"Status code: {response.status_code}")
        print(f"Response:\n{json.dumps(response.json(), indent=2)}")
        
        return response.json()
    except Exception as e:
        print(f"Error testing API: {str(e)}")
        return None

In [ ]:
# Test the API endpoint
# test_api_endpoint()

## Cleanup on Notebook Shutdown

In [ ]:
# Make sure to run this cell when you're done to clean up resources
# kill_all_processes()